# CGAN 실습

In [ ]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, metrics, optimizers
from tensorflow.keras.datasets import mnist

# MNIST 데이터 로드 및 전처리
def load_mnist_data():
    (X_train, y_train), (_, _) = mnist.load_data()
    X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
    X_train = (X_train - 127.5) / 127.5  # 이미지를 [-1, 1] 범위로 정규화
    y_train = tf.keras.utils.to_categorical(y_train, 10)  # One-hot encoding
    return X_train, y_train

X_train, y_train = load_mnist_data()


In [ ]:
class Hyperparameters:
    def __init__(self):
        self.image_size = (28, 28, 1)
        self.classes = 10
        self.batch_size = 32
        self.z_dim = 100
        self.learning_rate = 0.00005
        self.adam_beta_1 = 0.5
        self.adam_beta_2 = 0.999
        self.epochs = 30
        self.critic_steps = 5
        self.gp_weight = 10.0


In [ ]:
def make_generator_model(z_dim, num_classes=10):
    noise = layers.Input(shape=(z_dim,))
    label = layers.Input(shape=(num_classes,))
    x = layers.Concatenate()([noise, label])
    x = layers.Dense(7*7*256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = layers.Reshape((7, 7, 256))(x)
    x = layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')(x)
    model = models.Model([noise, label], x)
    model.summary()
    return model

In [ ]:
def make_discriminator_model(input_shape=(28, 28, 1), num_classes=10):
    image = layers.Input(shape=input_shape)
    label = layers.Input(shape=(num_classes,))
    label_img = layers.Dense(input_shape[0]*input_shape[1], activation='linear')(label)
    label_img = layers.Reshape((input_shape[0], input_shape[1], 1))(label_img)
    x = layers.Concatenate()([image, label_img])
    x = layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same')(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same')(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(1)(x)
    model = models.Model([image, label], x)
    model.summary()
    return model

In [ ]:
class ConditionalWGAN(models.Model):
    def __init__(self, generator, discriminator, latent_dim=100, critic_steps=5, gp_weight=10.0):
        super(ConditionalWGAN, self).__init__()
        self.generator = generator
        self.discriminator = discriminator
        self.latent_dim = latent_dim
        self.critic_steps = critic_steps
        self.gp_weight = gp_weight
        self.d_optimizer = optimizers.Adam(1e-4)
        self.g_optimizer = optimizers.Adam(1e-4)

    def compile(self):
        super(ConditionalWGAN, self).compile()
        self.d_loss_metric = metrics.Mean(name="d_loss")
        self.g_loss_metric = metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def gradient_penalty(self, batch_size, real_images, fake_images, labels):
        """Calculates the gradient penalty."""
        alpha = tf.random.normal([batch_size, 1, 1, 1], 0.0, 1.0)
        diff = fake_images - real_images
        interpolated = real_images + alpha * diff

        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated)
            pred = self.discriminator([interpolated, labels], training=True)

        grads = gp_tape.gradient(pred, [interpolated])[0]
        norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
        gp = tf.reduce_mean((norm - 1.0) ** 2)
        return gp

    def train_step(self, data):
        real_images, labels = data
        batch_size = tf.shape(real_images)[0]

        for _ in range(self.critic_steps):
            random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
            with tf.GradientTape() as tape:
                fake_images = self.generator([random_latent_vectors, labels], training=True)
                fake_logits = self.discriminator([fake_images, labels], training=True)
                real_logits = self.discriminator([real_images, labels], training=True)
                d_cost = tf.reduce_mean(fake_logits) - tf.reduce_mean(real_logits)
                gp = self.gradient_penalty(batch_size, real_images, fake_images, labels)
                d_loss = d_cost + gp * self.gp_weight

            d_gradient = tape.gradient(d_loss, self.discriminator.trainable_variables)
            self.d_optimizer.apply_gradients(zip(d_gradient, self.discriminator.trainable_variables))

        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        with tf.GradientTape() as tape:
            generated_images = self.generator([random_latent_vectors, labels], training=True)
            generated_logits = self.discriminator([generated_images, labels], training=True)
            g_loss = -tf.reduce_mean(generated_logits)

        g_gradient = tape.gradient(g_loss, self.generator.trainable_variables)
        self.g_optimizer.apply_gradients(zip(g_gradient, self.generator.trainable_variables))

        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)

        return {"d_loss": self.d_loss_metric.result(), "g_loss": self.g_loss_metric.result()}

In [ ]:
if not os.path.exists('output'):
    os.makedirs('output')

In [ ]:
params = Hyperparameters()
generator = make_generator_model(params.z_dim)
discriminator = make_discriminator_model(params.image_size)
cgan = ConditionalWGAN(generator, discriminator, params.z_dim, params.critic_steps, params.gp_weight)
cgan.compile()

# 데이터셋 준비
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1000).batch(params.batch_size)
# save_images_callback = SaveImagesCallback(generator, params.z_dim)

# 모델 훈련
cgan.fit(train_dataset, epochs=params.epochs)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 하이퍼파라미터 설정
z_dim = 100  # 또는 Hyperparameters 클래스에서 정의된 값을 사용하세요.

# 그리드 차원 설정
image_grid_rows = 10
image_grid_columns = 5

# 랜덤한 잡음 샘플링
z = np.random.normal(0, 1, (image_grid_rows * image_grid_columns, z_dim))

# 생성할 이미지 레이블 준비 (0-9까지 각 숫자를 5번씩 생성)
labels_to_generate = np.array([i for i in range(10) for _ in range(5)])
labels_to_generate = tf.keras.utils.to_categorical(labels_to_generate, 10)  # 원-핫 인코딩

# 랜덤한 잡음에서 이미지 생성
gen_imgs = generator.predict([z, labels_to_generate])

# 이미지 픽셀 값 [0, 1] 사이로 스케일 변환
gen_imgs = 0.5 * gen_imgs + 0.5

# 이미지 그리드 설정 및 출력
fig, axs = plt.subplots(image_grid_rows, image_grid_columns, figsize=(10, 20), sharey=True, sharex=True)

cnt = 0
for i in range(image_grid_rows):
    for j in range(image_grid_columns):
        axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
        axs[i, j].axis('off')
        axs[i, j].set_title(f"Digit: {np.argmax(labels_to_generate[cnt])}")  # 각 이미지의 레이블을 타이틀로 표시
        cnt += 1

plt.show()
